In [82]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import pandas as pd
import time

In [83]:
headers = {"User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.2.8) Gecko/20100722 Firefox/3.6.8 GTB7.1 (.NET CLR 3.5.30729)"}
base_url = "https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=0"

response = requests.get(base_url, headers = headers, timeout = 3 )

html = response.content
soup = BeautifulSoup(html, "lxml")

page_end = (int(soup.find("strong").text) // 15) + 1 

In [84]:
titles = []
urls = []
area = []
city = []
company = []
job_type = []

career_level = []
experience_needed = []
skills_and_tools = []

In [85]:
for page in range(0, page_end):
    
    time.sleep(1)
    
    base_url = f"https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start={page}"
    response = requests.get(base_url, headers=headers)
    
    if response.status_code == 200:            
        print('URL #{0}: {1}'.format(page+1, base_url))
    else:                                  
        print('Status code {0}: Skipping URL #{1}: {2}'.format(response.status_code, page+1, url))
        continue    
        
    html = response.content
    soup = BeautifulSoup(html, "lxml")
    
    #--------------------------------------------------------------------------------#
    
    anchors = soup.find_all('a', {"rel": "noreferrer", "class": "css-o171kl"})
    
    title = [title.string for title in anchors]
    titles.extend(title)
    
    absolute_url = [urljoin(base_url, link.get('href')) for link in anchors]
    urls.extend(absolute_url)
    
    #--------------------------------------------------------------------------------#
    
    location = soup.find_all('span', class_ = 'css-5wys0k')
    
    areas = [loc.text.split(',')[0].strip() for loc in location]
    area.extend(areas)
    
    cities = [loc.text.split(',')[1].strip() for loc in location]
    city.extend(cities)
    
    #--------------------------------------------------------------------------------#
    
    company_soup = soup.find_all('a', {"rel": "noreferrer", "class": "css-17s97q8"})
    
    companies = [comp.string.strip(' -') for comp in company_soup]
    company.extend(companies)
    
    #--------------------------------------------------------------------------------#
    
    job_types = soup.find_all('div', class_ = 'css-1lh32fc')
    
    for job in job_types:
        job_type.append(" | ".join([type.text.strip(" .") for type in job.find_all('span')]))
        
    #--------------------------------------------------------------------------------#
    
    detalis = soup.find_all('div', class_ = 'css-y4udm8')
    
    career_l = [d.find('a', class_ = 'css-o171kl').string for d in detalis]
    career_level.extend(career_l)
    
    experience = [d.find_all('div')[1].text.split('·',2)[1].strip() for d in detalis]
    experience_needed.extend(experience)
    
    for det in detalis:
        tools = det.find_all('a', class_ = 'css-5x9pm1')
        skills_and_tools.append(" | ".join([tool.text.strip(' ·') for tool in tools]))
        

URL #1: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=0
URL #2: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=1
URL #3: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=2
URL #4: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=3
URL #5: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=4
URL #6: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=5
URL #7: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=6
URL #8: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=7
URL #9: https://wuzzuf.net/search/jobs/?a=navbg&filters%5Bcountry%5D%5B0%5D=Egypt&q=Data-Analyst&start=8
URL #10: https://wuzzuf.net/search/jobs/?a=navbg&filter

In [65]:
dic_of_lists = {'titles': titles, 'area': area, 
                'city': city, 'company': company, 
                'job_type': job_type, 'career_level': career_level,
                'experience_needed': experience_needed, 'skills_and_tools': skills_and_tools, 'urls': urls}

data = pd.DataFrame(dic_of_lists)

data.to_csv('WUZZUF_WebScarping.csv')

In [78]:
data.head()

,titles,area,city,company,job_type,career_level,experience_needed,skills_and_tools,urls
0,Mid Senior Data Analyst,Maadi,Cairo,Worex,Full Time,Experienced,3+ Yrs of Exp,Data Analysis | Data Analyst | Information Tec...,https://wuzzuf.net/jobs/p/0T6a4KDMsFNn-Mid-Sen...
1,Junior Data Analyst,Maadi,Cairo,Worex,Full Time,Entry Level,1 - 2 Yrs of Exp,Analyst | Data Analyst | Data Analysis | Infor...,https://wuzzuf.net/jobs/p/iq5jLz08Y7Fb-Junior-...
2,Senior Data Analyst - Business Intelligence,Maadi,Cairo,Confidential,Full Time,Experienced,3 - 5 Yrs of Exp,Analyst | Business Administration | business |...,https://wuzzuf.net/jobs/p/FdG6WamryKOU-Senior-...
3,E-Commerce & Marketing Data Analyst,New Cairo,Cairo,Confidential,Full Time,Experienced,3 - 6 Yrs of Exp,Market Research | Data Analysis | Analysis | M...,https://wuzzuf.net/jobs/p/jzNIjX873nvy-E-Comme...
4,Data Analyst| MIS,Maadi,Cairo,GlobeMed Egypt,Full Time,Experienced,2 - 5 Yrs of Exp,MIS | Data Analytics | SAS | SQL | Tableau | A...,https://wuzzuf.net/jobs/p/UZjhvK3rVYOC-Data-An...
